In [360]:
#import tensorflow as tf

#from tensorflow.keras.layers import LSTM

# from tensorflow.keras.models import Sequential

#from tensorflow.keras.losses import MeanSquaredError
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# STBD DATA USE THESE TWO
#true_path_stbd = "D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\StbdTesting_1404_002_FINAL.csv"
#prediction_path = "D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\AIOutput_CSV_Files\\mb_predictions\\PredictedOutputTestMB13.csv"

#true_path_stbd = "D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\0001_1404.002NR_stbd_avgBelow75deg.csv"
true_path_stbd = "D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\0001_1404.038_stbd_FinalCleaned_RollReady.csv"
prediction_path = "D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\AIOutput_CSV_Files\\mb_predictions\\PredictedOutputTestMB36.csv"
#prediction_path = "D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\AIOutput_CSV_Files\\mb_predictions\\StbdTrueReference_038cleanedFinal.csv"


#training_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\\train1_port_new.csv"
#test_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\StbdTesting_1404_002.csv"
#test_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\\test1_stbd_new.csv"
df_true=pd.read_csv(true_path_stbd)
df_prediction=pd.read_csv(prediction_path)

In [361]:
# PORT DATA USE THESE TWO

#true_path_port = "D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\0001_1404.002NR_port_avgBelow75deg.csv"
true_path_port = "D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\0001_1404.038_port_FinalCleaned_RollReady.csv"
prediction_path = "D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\AIOutput_CSV_Files\\mb_predictions\\PredictedOutputTrainMB35.csv"
#prediction_path = "D:\\OneDrive\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\AIOutput_CSV_Files\\mb_predictions\\PortTrueReference_038cleanedFinal.csv"
df_true=pd.read_csv(true_path_port)
df_prediction=pd.read_csv(prediction_path)

In [ ]:
#df_true=pd.read_csv(true_path_stbd)
#df_true=pd.read_csv(true_path_port)
#df_prediction=pd.read_csv(prediction_path)


In [ ]:
df_prediction

In [362]:

#df_predictedDOA = df_prediction.iloc[:,4]
#df_prediction[["PredictedDOA"]] = df_prediction[["DOA"]]
df_predictedDOA = df_prediction[["PredictedDOA"]]
#df_predictedDOA = df_prediction[["DOA"]]
df_predictedDOA

,PredictedDOA
0,0.000000
1,0.000000
2,0.000000
3,0.000000
4,0.000000
...,...
3635094,69.889565
3635095,69.899101
3635096,69.903740
3635097,69.904716


In [363]:
df_reference = df_true[["DOA", "AngleUncertainty"]]
df_reference
df_reference['AngleUncertainty'].max()
df_reference['AngleUncertainty'].mean()
df_reference['AngleUncertainty'].var()
df_reference['DOA'].var()
df_comparison = df_predictedDOA.join(df_reference)
df_comparison
theShape = df_comparison.shape
numRows= theShape[0]
numCols = 4
#5504457 
df_results = pd.DataFrame(index=range(numRows),columns=range(numCols))
#df_results = pd.DataFrame({'PassFail'})
#df_comparison.insert()
df_results.columns = {"PassFail", "2STDPassFail", "4STDPassFail", "6STDPassFail"}
df_results
df_comparison = df_comparison.join(df_results)
df_comparison

,PredictedDOA,DOA,AngleUncertainty,PassFail,2STDPassFail,6STDPassFail,4STDPassFail
0,0.000000,-3.054000,21,NaN,NaN,NaN,NaN
1,0.000000,-0.054000,24,NaN,NaN,NaN,NaN
2,0.000000,0.486000,43,NaN,NaN,NaN,NaN
3,0.000000,1.404000,14,NaN,NaN,NaN,NaN
4,0.000000,1.530000,22,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
3635094,69.889565,73.770001,14,NaN,NaN,NaN,NaN
3635095,69.899101,73.626001,18,NaN,NaN,NaN,NaN
3635096,69.903740,73.626001,19,NaN,NaN,NaN,NaN
3635097,69.904716,73.578001,22,NaN,NaN,NaN,NaN


In [364]:
df_comparison['Delta'] = df_comparison['PredictedDOA'] - df_comparison['DOA']
df_comparison['AbsDelta'] = df_comparison['Delta'].abs()
print(df_comparison['Delta'].var())
df_comparison

6.123722581098001


,PredictedDOA,DOA,AngleUncertainty,PassFail,2STDPassFail,6STDPassFail,4STDPassFail,Delta,AbsDelta
0,0.000000,-3.054000,21,NaN,NaN,NaN,NaN,3.054000,3.054000
1,0.000000,-0.054000,24,NaN,NaN,NaN,NaN,0.054000,0.054000
2,0.000000,0.486000,43,NaN,NaN,NaN,NaN,-0.486000,0.486000
3,0.000000,1.404000,14,NaN,NaN,NaN,NaN,-1.404000,1.404000
4,0.000000,1.530000,22,NaN,NaN,NaN,NaN,-1.530000,1.530000
...,...,...,...,...,...,...,...,...,...
3635094,69.889565,73.770001,14,NaN,NaN,NaN,NaN,-3.880436,3.880436
3635095,69.899101,73.626001,18,NaN,NaN,NaN,NaN,-3.726899,3.726899
3635096,69.903740,73.626001,19,NaN,NaN,NaN,NaN,-3.722261,3.722261
3635097,69.904716,73.578001,22,NaN,NaN,NaN,NaN,-3.673284,3.673284


In [365]:
# less than (0.02*AngleUncertainty)/2, since this is the 2sigma value, and we want all values to be under 1 sigma.
df_comparison['PassFail'] = df_comparison['AbsDelta'] <= (0.02*df_comparison['AngleUncertainty']/2)
df_comparison['2STDPassFail'] = df_comparison['AbsDelta'] <= (0.02*df_comparison['AngleUncertainty'])
df_comparison['4STDPassFail'] = df_comparison['AbsDelta'] <= (2*0.02*df_comparison['AngleUncertainty'])
df_comparison['6STDPassFail'] = df_comparison['AbsDelta'] <= (3*0.02*df_comparison['AngleUncertainty'])
df_comparison

,PredictedDOA,DOA,AngleUncertainty,PassFail,2STDPassFail,6STDPassFail,4STDPassFail,Delta,AbsDelta
0,0.000000,-3.054000,21,False,False,False,False,3.054000,3.054000
1,0.000000,-0.054000,24,True,True,True,True,0.054000,0.054000
2,0.000000,0.486000,43,False,True,True,True,-0.486000,0.486000
3,0.000000,1.404000,14,False,False,False,False,-1.404000,1.404000
4,0.000000,1.530000,22,False,False,False,False,-1.530000,1.530000
...,...,...,...,...,...,...,...,...,...
3635094,69.889565,73.770001,14,False,False,False,False,-3.880436,3.880436
3635095,69.899101,73.626001,18,False,False,False,False,-3.726899,3.726899
3635096,69.903740,73.626001,19,False,False,False,False,-3.722261,3.722261
3635097,69.904716,73.578001,22,False,False,False,False,-3.673284,3.673284


In [366]:
PassCount = df_comparison['PassFail'].sum()
PassCount2STD = df_comparison['2STDPassFail'].sum()
PassCount4STD = df_comparison['4STDPassFail'].sum()
PassCount6STD = df_comparison['6STDPassFail'].sum()

PassCount, PassCount2STD, PassCount4STD, PassCount6STD

(140884, 308374, 700694, 1122536)

In [367]:
FailCount = (~df_comparison['PassFail']).sum()
FailCount2STD = (~df_comparison['2STDPassFail']).sum()
FailCount4STD = (~df_comparison['4STDPassFail']).sum()
FailCount6STD = (~df_comparison['6STDPassFail']).sum()

FailCount, FailCount2STD, FailCount4STD, FailCount6STD

(3494215, 3326725, 2934405, 2512563)

In [368]:
TotalCount = PassCount+FailCount
TotalCount2STD = PassCount2STD+FailCount2STD
TotalCount4STD = PassCount4STD+FailCount4STD
TotalCount6STD = PassCount6STD+FailCount6STD

TotalCount, TotalCount2STD, TotalCount4STD, TotalCount6STD

(3635099, 3635099, 3635099, 3635099)

In [369]:
PercentPass = 100*(PassCount/TotalCount)
PercentPass2STD = 100*(PassCount2STD/TotalCount2STD)
PercentPass4STD = 100*(PassCount4STD/TotalCount4STD)
PercentPass6STD = 100*(PassCount6STD/TotalCount6STD)

PercentPass, PercentPass2STD, PercentPass4STD, PercentPass6STD

(3.8756578569111877, 8.483235257141553, 19.275788637393372, 30.880479458743764)

In [ ]:
print('Maximum error within 1 STD Deviation: ', 0.5*0.02*df_comparison['AngleUncertainty'].max(), 'degrees')
print('Minimun error within 1 STD Deviation: ', 0.5*0.02*df_comparison['AngleUncertainty'].min(), 'degrees')